In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_trunc, sum, count, col
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, DoubleType
from collections import namedtuple
from datetime import datetime


In [44]:
# Create a SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [45]:
# Load data
df_green = spark.read.parquet('data/pq/green/*/*')


In [46]:
# Define start date
start = datetime(year=2020, month=1, day=1)

In [47]:
# Define a schema for the result
result_schema = StructType([
    StructField('hour', TimestampType(), True),
    StructField('zone', IntegerType(), True),
    StructField('revenue', DoubleType(), True),
    StructField('count', IntegerType(), True)
])

In [56]:
# Define a namedtuple for df_predicts
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [67]:
rdd = df_green.rdd.filter(lambda row: row['lpep_pickup_datetime'] >= start) \
    .map(lambda row: ((row['lpep_pickup_datetime'].replace(minute=0, second=0, microsecond=0), row['PULocationID']), (row['total_amount'], 1))) \
#     .reduceByKey(lambda left_value, right_value: (left_value[0] + right_value[0], left_value[1] + right_value[1]))

In [68]:
# Reduce by key to calculate revenue
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount, output_count)

In [70]:
reduced_rdd = rdd.reduceByKey(calculate_revenue, numPartitions=4, partitionFunc=portable_hash, serializer=PickleSerializer())

NameError: name 'portable_hash' is not defined

In [52]:
def unwrap(row):
    return PredictedRow(
        VendorID=row[0][0], 
        lpep_pickup_datetime=row[0][0], 
        PULocationID=row[0][1], 
        DOLocationID=None, 
        trip_distance=None, 
        predicted_duration=row[1][0]
    )

In [55]:
df_result = df_green \
    .filter(col('lpep_pickup_datetime') >= start) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

AttributeError: 'DataFrame' object has no attribute 'map'